In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import spacy,os
import argparse
import re
from tqdm import tqdm
from collections import OrderedDict
import string
import numpy as np
from spacy.lang.en import English
import time
nl = English()
import sys
import pandas as pd
import argparse
parser = argparse.ArgumentParser()

In [2]:
repeat = 5
data = []
doc = []
l3 = []
summary = []
hypothesis = ""
word_count = []
pair_similarity = []
summary_string = []

In [3]:
def count_word(index):
    global doc
    Doc = nl(doc[index])
    tokens = [t.text for t in Doc]
    tokens = [t for t in tokens if len(t.translate(t.maketrans('', '', string.punctuation + string.whitespace))) > 0] # + string.digits
    return len(tokens)

In [4]:
def store_word_count():
    global word_count,doc
    word_count = []
    for i in range(0,len(doc)):
        word_count.append(count_word(i))

In [5]:
def maximum(index, toPrint=0):
    global summary, pair_similarity
    length = len(summary)
    if(length!=0):
        max=0
        for i in range(length):
            a=pair_similarity[index][summary[i]]
            if(a>max):
                max=a
            if toPrint:
              print(str(summary[i])+" -> "+str(a))
        return max
    else:
        return 0

In [6]:
def count_sum(summary):
    sum=0
    length = len(summary)
    for i in range(length):
        sum+=count_word(summary[i])
    return sum

In [7]:
def mmr_sorted(lambda_, doc, length):
    global word_count, pair_similarity, summary
    #print('Inside MMR')
    print(length)
    l3 = []
    vectorizer = TfidfVectorizer(smooth_idf=False)
    X = vectorizer.fit_transform(doc)
    y = X.toarray()
    rows = y.shape[0]
    cols = y.shape[1]
    pair_similarity = []
    for i in range(rows):
        max=-1
        pair_similarity.append([])
        for j in range(rows):
            if(j!=i):
                a = np.sum(np.multiply(y[i],y[j]))
                pair_similarity[-1].append(a)
                if(a>max):
                    max=a
            else:
                pair_similarity[-1].append(1)
        l3.append(max)
    store_word_count()
    l = len(doc)  
    count = 0
    last = -1
    summary = []
    summary_word_count = 0
    while(1):
        if (summary_word_count < length):
            max=-1
            for i in range(l):
                a = maximum(i)
                mmrscore = lambda_*l3[i] - (1-lambda_)*a
                if(mmrscore >= max):
                    max = mmrscore
                    ind = i
            summary.append(ind)
            summary_word_count += word_count[ind]
        else:
            #print('Bye')
            break

In [8]:
def listToString():  
    global summary_string, word_count, hypothesis, summary, doc
    summary_string = []
    leng = 0
    for i in summary:
      if doc[i] not in summary_string:
          summary_string.append(doc[i])
          leng += word_count[i]
    hypothesis = "".join(summary_string) 

In [9]:
import pandas as pd
import os

dataset = 'IN-Abs'   
mode = 'test-data'       

base_path = os.path.abspath(os.path.join(os.getcwd(), "../../"))
stats_path = os.path.join(base_path, f"data/{dataset}/{mode}/stats-IN-test.txt")
length_path = os.path.join(base_path, f"data/{dataset}/{mode}/length.txt")

df = pd.read_csv(stats_path, sep="\t", header=None)
df[[0, 4]].to_csv(length_path, sep="\t", header=False, index=False)

print("✅ Created:", length_path)


✅ Created: /Users/abhinavmittal/Desktop/minor/data/IN-Abs/test-data/length.txt


In [10]:
data_path = '../../data/in-abs/train/judgement'
summary_path = '../../data/in-abs/train/summary'
length_file = '../../data/in-abs/train/length.txt'


In [11]:
import os
import pandas as pd

output_path = os.getcwd()

data_path = '../../data/in-abs/test-data/judgement'     
length_file = '../../data/in-abs/test-data/length.txt'  

X1 = pd.read_csv(length_file, sep="\t", header=None)

filename = '78.txt'

index = X1[X1[0] == filename].index[0]
length1 = 400

doc = []
with open(os.path.join(data_path, filename), 'r') as file:
    for x in file:
        if x.strip():  
            doc.append(x)

lamda = 0.6
mmr_sorted(lamda, doc, length1)
listToString()

output_file_path = os.path.join(output_path, filename)
with open(output_file_path, "w+") as f:
    f.write(hypothesis)

print(f"✅ Summary saved as: {output_file_path}")


400
✅ Summary saved as: /Users/abhinavmittal/Desktop/minor/model/mmr/78.txt
